<a href="https://colab.research.google.com/github/Doggo21345/Movies_Bechdel/blob/main/Movies_Side_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA analysis

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re

In [2]:
bechdel_url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2021/2021-03-09/raw_bechdel.csv"
movies_url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2021/2021-03-09/movies.csv"

raw_bechdel = pd.read_csv(bechdel_url)
movies = pd.read_csv(movies_url)

In [6]:
movies.head(10)

,year,imdb,title,test,clean_test,binary,budget,domgross,intgross,code,...,director,released,actors,genre,awards,runtime,type,poster,imdb_votes,error
0,2013,tt1711425,21 &amp; Over,notalk,notalk,FAIL,13000000,25682380.0,42195766.0,2013FAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,tt1343727,Dredd 3D,ok-disagree,ok,PASS,45000000,13414714.0,40868994.0,2012PASS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,tt2024544,12 Years a Slave,notalk-disagree,notalk,FAIL,20000000,53107035.0,158607035.0,2013FAIL,...,Steve McQueen,08 Nov 2013,"Chiwetel Ejiofor, Dwight Henry, Dickie Gravois...","Biography, Drama, History",Won 3 Oscars. Another 131 wins & 137 nominations.,134 min,movie,http://ia.media-imdb.com/images/M/MV5BMjExMTEz...,"143,446",NaN
3,2013,tt1272878,2 Guns,notalk,notalk,FAIL,61000000,75612460.0,132493015.0,2013FAIL,...,Baltasar Kormákur,02 Aug 2013,"Denzel Washington, Mark Wahlberg, Paula Patton...","Action, Comedy, Crime",1 win.,109 min,movie,http://ia.media-imdb.com/images/M/MV5BNTQ5MTgz...,"87,301",NaN
4,2013,tt0453562,42,men,men,FAIL,40000000,95020213.0,95020213.0,2013FAIL,...,Brian Helgeland,12 Apr 2013,"Chadwick Boseman, Harrison Ford, Nicole Behari...","Biography, Drama, Sport",3 wins & 13 nominations.,128 min,movie,http://ia.media-imdb.com/images/M/MV5BMTQwMDU4...,"43,608",NaN
5,2013,tt1335975,47 Ronin,men,men,FAIL,225000000,38362475.0,145803842.0,2013FAIL,...,Carl Rinsch,25 Dec 2013,"Keanu Reeves, Hiroyuki Sanada, Ko Shibasaki, T...","Action, Adventure, Fantasy",1 nomination.,118 min,movie,http://ia.media-imdb.com/images/M/MV5BMTc0MjE2...,"25,735",NaN
6,2013,tt1606378,A Good Day to Die Hard,notalk,notalk,FAIL,92000000,67349198.0,304249198.0,2013FAIL,...,John Moore,14 Feb 2013,"Bruce Willis, Jai Courtney, Sebastian Koch, Ma...","Action, Crime, Thriller",2 wins & 3 nominations.,98 min,movie,http://ia.media-imdb.com/images/M/MV5BMTcwNzgy...,"123,837",NaN
7,2013,tt2194499,About Time,ok-disagree,ok,PASS,12000000,15323921.0,87324746.0,2013PASS,...,Richard Curtis,08 Nov 2013,"Domhnall Gleeson, Rachel McAdams, Bill Nighy, ...","Drama, Fantasy, Romance",1 win & 6 nominations.,123 min,movie,http://ia.media-imdb.com/images/M/MV5BMTA1ODUz...,"85,871",NaN
8,2013,tt1814621,Admission,ok,ok,PASS,13000000,18007317.0,18007317.0,2013PASS,...,Paul Weitz,22 Mar 2013,"Tina Fey, Ann Harada, Ben Levin, Dan Levy","Comedy, Drama, Romance",NaN,107 min,movie,http://ia.media-imdb.com/images/M/MV5BOTE2OTkw...,"18,973",NaN
9,2013,tt1815862,After Earth,notalk,notalk,FAIL,130000000,60522097.0,244373198.0,2013FAIL,...,M. Night Shyamalan,31 May 2013,"Jaden Smith, Will Smith, Sophie Okonedo, Zoë K...","Action, Adventure, Sci-Fi",3 wins & 5 nominations.,100 min,movie,http://ia.media-imdb.com/images/M/MV5BMTY3MzQy...,"108,264",NaN


In [8]:
movies.columns

Index(['year', 'imdb', 'title', 'test', 'clean_test', 'binary', 'budget',
       'domgross', 'intgross', 'code', 'budget_2013', 'domgross_2013',
       'intgross_2013', 'period_code', 'decade_code', 'imdb_id', 'plot',
       'rated', 'response', 'language', 'country', 'writer', 'metascore',
       'imdb_rating', 'director', 'released', 'actors', 'genre', 'awards',
       'runtime', 'type', 'poster', 'imdb_votes', 'error'],
      dtype='object')

In [13]:
movies = movies.rename(columns={"binary": "test_score"})


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1794 entries, 0 to 1793
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           1794 non-null   int64  
 1   imdb           1794 non-null   object 
 2   title          1794 non-null   object 
 3   test           1794 non-null   object 
 4   clean_test     1794 non-null   object 
 5   binary         1794 non-null   object 
 6   budget         1794 non-null   int64  
 7   domgross       1777 non-null   float64
 8   intgross       1783 non-null   float64
 9   code           1794 non-null   object 
 10  budget_2013    1794 non-null   int64  
 11  domgross_2013  1776 non-null   float64
 12  intgross_2013  1783 non-null   float64
 13  period_code    1615 non-null   float64
 14  decade_code    1615 non-null   float64
 15  imdb_id        1794 non-null   object 
 16  plot           1592 non-null   object 
 17  rated          1582 non-null   object 
 18  response

In [3]:
rating_map = {
    0 : "Unscored",
    1 : "One names women",
    2 : "Two women talk",
    3 : "Two women don't talk about men"
}

raw_bechdel["rating_text"] = raw_bechdel["rating"].map(rating_map)


This line is meant to help clean out the varibales

What we do with this code is


1.   First we create a **dictionary** since they are best suited with map to create the actual descriptions of the bechdel ratings
2.   Then we use the map function to actually map the rating variable to a new column called the **"rating_text"**



In [5]:
movies['imdb_id'] = movies.imdb.str.replace('tt', '')

In [ ]:
df_new = 'year', 'imdb', 'title', 'test', 'clean_test', 'test_score', 'budget',
       'domgross', 'intgross', 'code', 'budget_2013', 'domgross_2013',
       'intgross_2013', #'period_code', 'decade_code',
       'imdb_id', 'plot',
       'rated', 'response', 'language', 'country', 'writer', 'metascore',
       'imdb_rating', 'director', 'released', 'actors', 'genre', 'awards',
       'runtime', 'type', 'poster', 'imdb_votes', 'error'],
      dtype='object'